In [0]:
import numpy
import pandas as pd
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
import keras
import sklearn
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
seed = 7
numpy.random.seed(seed)

In [0]:
help(pd.read_csv)

In [0]:
# load training dataset
data = pd.read_csv('drive/My Drive/MMA/RSM 8431 Analytics Colloquia/Bayesian '+
                      'Network Assignment/LearningData.csv', sep=';')
data.loc[data['Class'] == 'Benign', 'Class'] = 0
data.loc[data['Class'] == 'Malignant', 'Class'] = 1
data = data.iloc[:,1:]
data['Bare Nuclei'] = pd.to_numeric(data['Bare Nuclei'],errors='coerce',downcast='integer')
data['Bare Nuclei'] = (data['Bare Nuclei'].fillna(int(data['Bare Nuclei'].mean()))).astype('int64')
data

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...
555,3,1,1,1,2,1,1,1,1,0
556,3,1,1,1,2,1,2,1,2,0
557,2,1,1,1,2,1,1,1,1,0
558,4,8,6,4,3,4,10,6,1,1


In [0]:
# load test dataset
data2 = pd.read_csv('drive/My Drive/MMA/RSM 8431 Analytics Colloquia/Bayesian '+
                      'Network Assignment/TestData.csv', sep=';')
data2.loc[data2['Class'] == 'Benign', 'Class'] = 0
data2.loc[data2['Class'] == 'Malignant', 'Class'] = 1
data2 = data2.iloc[:,1:]
data2['Bare Nuclei'] = pd.to_numeric(data2['Bare Nuclei'],errors='coerce',downcast='integer')
data2['Bare Nuclei'] = (data2['Bare Nuclei'].fillna(int(data2['Bare Nuclei'].mean()))).astype('int64')
data2

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,2,1,2,1,2,1,3,1,1,0
1,4,2,1,1,2,1,2,1,1,0
2,5,4,4,9,2,10,5,6,1,1
3,6,10,10,2,8,10,7,3,3,1
4,10,3,6,2,3,5,4,10,2,1
...,...,...,...,...,...,...,...,...,...,...
134,5,10,10,8,5,5,7,10,1,1
135,1,1,1,1,2,1,1,1,1,0
136,3,1,1,1,2,1,2,3,1,0
137,3,1,1,1,3,2,1,1,1,0


In [0]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 10 columns):
Clump Thickness                139 non-null int64
Uniformity of Cell Size        139 non-null int64
Uniformity of Cell Shape       139 non-null int64
Marginal Adhesion              139 non-null int64
Single Epithelial Cell Size    139 non-null int64
Bare Nuclei                    139 non-null int64
Bland Chromatin                139 non-null int64
Normal Nucleoli                139 non-null int64
Mitoses                        139 non-null int64
Class                          139 non-null int64
dtypes: int64(10)
memory usage: 11.0 KB


In [0]:
# no train test split for this example
# scale
scaler = preprocessing.MinMaxScaler()

In [0]:
scaled_data = scaler.fit_transform(data)
scaled_data
scaled_data2 = scaler.fit_transform(data2)
scaled_data2

array([[0.11111111, 0.        , 0.11111111, ..., 0.        , 0.        ,
        0.        ],
       [0.33333333, 0.11111111, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.44444444, 0.33333333, 0.33333333, ..., 0.55555556, 0.        ,
        1.        ],
       ...,
       [0.22222222, 0.        , 0.        , ..., 0.22222222, 0.        ,
        0.        ],
       [0.22222222, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.44444444, 1.        , 1.        , ..., 1.        , 0.11111111,
        1.        ]])

In [0]:
scaled_data = pd.DataFrame(scaled_data)
scaled_data2 = pd.DataFrame(scaled_data2)
scaled_data

,0,1,2,3,4,5,6,7,8,9
0,0.444444,0.000000,0.000000,0.000000,0.111111,0.000000,0.222222,0.000000,0.000000,0.0
1,0.444444,0.333333,0.333333,0.444444,0.666667,1.000000,0.222222,0.111111,0.000000,0.0
2,0.222222,0.000000,0.000000,0.000000,0.111111,0.111111,0.222222,0.000000,0.000000,0.0
3,0.555556,0.777778,0.777778,0.000000,0.222222,0.333333,0.222222,0.666667,0.000000,0.0
4,0.333333,0.000000,0.000000,0.222222,0.111111,0.000000,0.222222,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
555,0.222222,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0
556,0.222222,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.111111,0.0
557,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0
558,0.333333,0.777778,0.555556,0.333333,0.222222,0.333333,1.000000,0.555556,0.000000,1.0


In [0]:
# split predictors from target
x_train = scaled_data.iloc[:,0:9]
y_train = scaled_data.iloc[:,9]
x_test = scaled_data2.iloc[:,0:9]
y_test = scaled_data2.iloc[:,9]

In [0]:
# create model
model = Sequential()
model.add(Dense(9, input_dim=9, activation="relu"))
model.add(Dense(9, input_dim=9, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(lr=0.00012), metrics=["accuracy"])

In [0]:
model.fit(x_train, y_train, epochs=80, batch_size=5, verbose=1)

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier

In [0]:
# function to create model
def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=9, activation="relu"))
    model.add(Dense(neurons, input_dim=9, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(lr=0.00012), metrics=["accuracy"])
    return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=80, batch_size=5, verbose=1)

In [0]:
# grid search parameters
neurons = [4,5,6,7,8,9,10,11]
param_grid = dict(neurons = neurons)

In [0]:
grid = GridSearchCV(estimator = model, param_grid= param_grid)

In [0]:
grid_result = grid.fit(x_train,y_train)

In [0]:
means = grid_result.cv_results_["mean_test_score"]

In [0]:
stds = grid_result.cv_results_["std_test_score"]

In [0]:
parms = grid_result.cv_results_["params"]

In [0]:
for mean, stdev, param in zip(means, stds, parms):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.951786 (0.029124) with: {'neurons': 4}
0.955357 (0.041496) with: {'neurons': 5}
0.958929 (0.038548) with: {'neurons': 6}
0.958929 (0.039770) with: {'neurons': 7}
0.957143 (0.043522) with: {'neurons': 8}
0.962500 (0.033120) with: {'neurons': 9}
0.957143 (0.034534) with: {'neurons': 10}
0.962500 (0.027894) with: {'neurons': 11}


In [0]:
# Check Results
y_pred = model.predict(x_test)
y_pred = [round(x[0]) for x in y_pred]
conf_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred, normalize='true')*100
conf_matrix = pd.DataFrame(conf_matrix).transpose().rename(
    columns = {0:'Act-Benign (87)', 1:'Act-Mal (52)'}, index = {0:'Pred-Benign (84)', 1:'Pred-Mal (55)'}).round(2)
conf_matrix = conf_matrix.astype(str) + '%'
conf_matrix

,Act-Benign (87),Act-Mal (52)
Pred-Benign (84),95.4%,1.92%
Pred-Mal (55),4.6%,98.08%
